In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
#https://drive.google.com/file/d/183B_W_R2qSnT_JjEUcI3Ny1vH36desCp/view?usp=sharing
file1 = drive.CreateFile({'id':'183B_W_R2qSnT_JjEUcI3Ny1vH36desCp'}) # replace the id with id of file you want to access
file1.GetContentFile('credit_train.csv')

In [9]:
#https://drive.google.com/file/d/1ceLPls-2L2RuPoAJXAwGgcqxufoHAs0t/view?usp=sharing
file2 = drive.CreateFile({'id':'1ceLPls-2L2RuPoAJXAwGgcqxufoHAs0t'}) # replace the id with id of file you want to access
file2.GetContentFile('credit_test.csv')

In [10]:
import pandas as pd
import numpy as np

In [53]:
df_train = pd.read_csv('credit_train.csv')

In [54]:
df_train.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


In [55]:
df_train.info()
#non numerica type columns: Loan_ID, Customer ID, Loan Status, Term, Years in Current job, Home Ownership, Purpose

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100514 entries, 0 to 100513
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       100000 non-null  object 
 1   Customer ID                   100000 non-null  object 
 2   Loan Status                   100000 non-null  object 
 3   Current Loan Amount           100000 non-null  float64
 4   Term                          100000 non-null  object 
 5   Credit Score                  80846 non-null   float64
 6   Annual Income                 80846 non-null   float64
 7   Years in current job          95778 non-null   object 
 8   Home Ownership                100000 non-null  object 
 9   Purpose                       100000 non-null  object 
 10  Monthly Debt                  100000 non-null  float64
 11  Years of Credit History       100000 non-null  float64
 12  Months since last delinquent  46859 non-null

In [56]:
len(df_train)

100514

In [57]:
#check missing values 
df_train.isna().sum()

Loan ID                           514
Customer ID                       514
Loan Status                       514
Current Loan Amount               514
Term                              514
Credit Score                    19668
Annual Income                   19668
Years in current job             4736
Home Ownership                    514
Purpose                           514
Monthly Debt                      514
Years of Credit History           514
Months since last delinquent    53655
Number of Open Accounts           514
Number of Credit Problems         514
Current Credit Balance            514
Maximum Open Credit               516
Bankruptcies                      718
Tax Liens                         524
dtype: int64

In [58]:
print(df_train['Months since last delinquent'].isna().sum()/len(df_train))

0.5338062359472312


In [59]:
type(df_train['Months since last delinquent'])

pandas.core.series.Series

In [60]:
#Overall, we need to impute missing values for all the columns, since missing values for Months since last delinquent take up over 50% 
import numpy as np
from sklearn.impute import SimpleImputer

In [62]:
#impute missing values for numerical data : mean
# Select numeric columns.
numeric = df_train.select_dtypes('number')
# Select string and object columns.
categoric = df_train.select_dtypes('object')

# Fill numeric columns with mean.
df_train[numeric.columns] = numeric.fillna(numeric.mean())
# Fill object columns with mode.
df_train[categoric.columns] = categoric.fillna(categoric.agg(lambda x: x.mode().values[0]))

In [63]:
#double check missing values
df_train.isna().sum()

Loan ID                         0
Customer ID                     0
Loan Status                     0
Current Loan Amount             0
Term                            0
Credit Score                    0
Annual Income                   0
Years in current job            0
Home Ownership                  0
Purpose                         0
Monthly Debt                    0
Years of Credit History         0
Months since last delinquent    0
Number of Open Accounts         0
Number of Credit Problems       0
Current Credit Balance          0
Maximum Open Credit             0
Bankruptcies                    0
Tax Liens                       0
dtype: int64

In [65]:
df_train.shape

(100514, 19)

In [64]:
#check duplicate values
df_train.duplicated

<bound method DataFrame.duplicated of                                      Loan ID  ... Tax Liens
0       14dd8831-6af5-400b-83ec-68e61888a048  ...  0.000000
1       4771cc26-131a-45db-b5aa-537ea4ba5342  ...  0.000000
2       4eed4e6a-aa2f-4c91-8651-ce984ee8fb26  ...  0.000000
3       77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a  ...  0.000000
4       d4062e70-befa-4995-8643-a0de73938182  ...  0.000000
...                                      ...  ...       ...
100509  00069ff1-a877-4d35-81be-7cd359b99956  ...  0.029313
100510  00069ff1-a877-4d35-81be-7cd359b99956  ...  0.029313
100511  00069ff1-a877-4d35-81be-7cd359b99956  ...  0.029313
100512  00069ff1-a877-4d35-81be-7cd359b99956  ...  0.029313
100513  00069ff1-a877-4d35-81be-7cd359b99956  ...  0.029313

[100514 rows x 19 columns]>

In [66]:
#check unique values for each column
df_train.nunique()
#meaningful columns often have fewer number of unique colunmns
#we need to conduct some transformation for those columns which are categorical 

Loan ID                         81999
Customer ID                     81999
Loan Status                         2
Current Loan Amount             22005
Term                                2
Credit Score                      325
Annual Income                   36175
Years in current job               11
Home Ownership                      4
Purpose                            16
Monthly Debt                    65766
Years of Credit History           507
Months since last delinquent      117
Number of Open Accounts            52
Number of Credit Problems          15
Current Credit Balance          32731
Maximum Open Credit             44597
Bankruptcies                        9
Tax Liens                          13
dtype: int64

In [67]:
df_train[categoric.columns].nunique()

Loan ID                 81999
Customer ID             81999
Loan Status                 2
Term                        2
Years in current job       11
Home Ownership              4
Purpose                    16
dtype: int64

In [85]:
#feature encoding for categorical varaibles
#label encoding: convert each value in a column to a number 
#All the categorical variables except years in current job can be handled with label encoding 
#Years in current job are required to be transformed into numerical variables
#create initial categorical dataframe
categoric_cols = list(categoric.columns)
categoric_cols.remove('Years in current job')
categoric_cols

['Loan ID', 'Customer ID', 'Loan Status', 'Term', 'Home Ownership', 'Purpose']

In [86]:
#creatin instance of labelencoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train[categoric_cols] = df_train[categoric_cols].apply(le.fit_transform)

In [87]:
df_train.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6697,48724,1,445412.0,1,709.000000,1.167493e+06,8 years,1,5,5214.74,17.2,34.901321,6.0,1.0,228190.0,416746.0,1.0,0.0
1,23041,14688,1,262328.0,1,1076.456089,1.378277e+06,10+ years,1,3,33295.98,21.1,8.000000,35.0,0.0,229976.0,850784.0,0.0,0.0
2,25441,30541,1,99999999.0,1,741.000000,2.231892e+06,8 years,2,3,29200.53,14.9,29.000000,18.0,1.0,297996.0,750090.0,0.0,0.0
3,38341,74173,1,347666.0,0,721.000000,8.069490e+05,3 years,2,3,8741.90,12.0,34.901321,9.0,0.0,256329.0,386958.0,0.0,0.0
4,68058,41470,1,176220.0,1,1076.456089,1.378277e+06,5 years,3,3,20639.70,6.1,34.901321,15.0,0.0,253460.0,427174.0,0.0,0.0
